In [5]:
import random
import re
from os import sep

import numpy as np
import pandas as pd
import requests
import tmdbsimple as tmdb
from scipy.stats import pearsonr
from surprise import SVD, Dataset, KNNBasic, Reader
from surprise.model_selection import cross_validate
from collections import defaultdict, Counter
from itertools import islice
import collections

In [2]:
tmdb.REQUESTS_SESSION = requests.Session()
tmdb.API_KEY = '1e11e7d4c5f3aad6e459fc0f63bfb0f5'
tmdb.REQUESTS_TIMEOUT = 5

In [3]:
users_df = pd.read_csv(
    "../data/users.txt", names=["user_id", "age", "gender", "occupation"], sep="\t"
)

In [4]:
users_df

,user_id,age,gender,occupation
0,1,24,M,technician
1,2,53,F,other
2,3,23,M,writer
3,4,24,M,technician
4,5,33,F,other
...,...,...,...,...
938,939,26,F,student
939,940,32,M,administrator
940,941,20,M,student
941,942,48,F,librarian


In [107]:
def get_user_type(gender, age, occupation):
    res = 0
    group1 = ['doctor', 'healthcare', 'entertainment']
    group2 = ['engineer', 'programmer', 'scientist', 'technician']
    group3 = ['artist', 'writer', 'librarian', 'homemaker']
    group4 = ['none', 'other', 'student', 'retired']
    group5 = ['executive', 'lawyer', 'administrator', 'salesman', 'marketing']

    if occupation in group1:
        if age < 23:
            res = 1 if gender == 'M' else 2
        elif 24 < age < 40:
            res = 3 if gender == 'M' else 4
        elif 41 < age < 60:
            res = 5 if gender == 'M' else 6
        else:
            res = 7 if gender == 'M' else 8
    elif occupation in group2:
        if age < 23:
            res = 9 if gender == 'M' else 9
        elif 24 < age < 40:
            res = 11 if gender == 'M' else 12
        elif 41 < age < 60:
            res = 13 if gender == 'M' else 14
        else:
            res = 15 if gender == 'M' else 16
    elif occupation in group3:
        if age < 23:
            res = 17 if gender == 'M' else 18
        elif 24 < age < 40:
            res = 19 if gender == 'M' else 20
        elif 41 < age < 60:
            res = 21 if gender == 'M' else 22
        else:
            res = 23 if gender == 'M' else 24
    elif occupation in group4:
        if age < 23:
            res = 25 if gender == 'M' else 26
        elif 24 < age < 40:
            res = 27 if gender == 'M' else 28
        elif 41 < age < 60:
            res = 29 if gender == 'M' else 30
        else:
            res = 31 if gender == 'M' else 32
    elif occupation in group5:
        if age < 23:
            res = 33 if gender == 'M' else 34
        elif 24 < age < 40:
            res = 35 if gender == 'M' else 36
        elif 41 < age < 60:
            res = 37 if gender == 'M' else 38
        else:
            res = 39 if gender == 'M' else 40

    return res - 1 if res > 9 else res

def movie_votes_demographic(grupos, ratings, movie_id, grupo):
    usuarios = [k for k, v in grupos.items() if v == grupo]
    return len(ratings[(ratings.user_id.isin(usuarios)) & (ratings.movie_id == movie_id)].index), ratings[(ratings.user_id.isin(usuarios)) & (ratings.movie_id == movie_id)].mean()['rating'], ratings[ratings.movie_id == movie_id].mean()['rating']

def genre_seen(films_df, ratings, user_id, genre_name):
    scores = []
    for film_id in ratings[ratings['user_id'] == user_id].movie_id.tolist():
        if films_df[films_df['movie_id'] == film_id][genre_name].tolist()[0] == 1:
            scores.append(ratings[(ratings['movie_id'] == film_id) & (ratings['user_id'] == user_id)]['rating'].tolist()[0])

    return sum(scores)/len(scores) if scores != [] else 0

def take(n, iterable):
    res = {}
    i = 0
    for k, v in iterable.items():
        if i >= n:
            break
        res[k] = v
        i += 1
        
    return res

def get_genres_score(users, generos, films_df, ratings):
    best_genres = {}
    for u in users:
        for i in generos['genre_name'].tolist():
            puntos = genre_seen(films_df, ratings, u, i)
            if puntos > 2.5:
                if i not in best_genres.keys():
                    best_genres[i] = puntos/len(users)
                else:
                    best_genres[i] += puntos/len(users)

    best_genres = {k: v for k, v in sorted(best_genres.items(), key=lambda item: item[1])}
    best_genres = take(6, best_genres)

    pref = []

    for genero in generos['genre_name'].tolist():
        add = False
        for k, v in best_genres.items():
            if genero == k:
                add = True
                break
        if add:
            pref.append(1)
        else:
            pref.append(0)

    return pref
    

def obtener_vecinos(preferencias, user, k=1):
    vecinos = [0]*k
    vecinos_score = [0]*k
    pref = preferencias[user]
    for i in range(0, preferencias.shape[0]):
        if i == user:
            continue
        pref_comp = np.matrix([pref, preferencias[i]])
        score = sum([1 if pref_comp[0,j] == pref_comp[1,j] else 0 for j in range(0, pref_comp.shape[1])])
        if score > min(vecinos_score):
            vecinos[vecinos_score.index(min(vecinos_score))] = i + 1
            vecinos_score[vecinos_score.index(min(vecinos_score))] = score
            
            
    return vecinos, vecinos_score

def get_dg_pref(dg, generos, films_df, ratings):
    res = []
    for x in set(dg.values()):
        u_dg = [k for k, v in dg.items() if v == x]
        scores = get_genres_score(u_dg, generos, films_df, ratings)
        res.append(scores)

    return np.matrix(res)
        



In [5]:
grupos_demograficos = {}
for user_id in users_df.user_id.unique().tolist():
    user = users_df[users_df.user_id == user_id]
    user_demo_group = get_user_type(user.gender.tolist()[0], user.age.tolist()[0], user.occupation.tolist()[0])
    grupos_demograficos[user_id] = user_demo_group

In [6]:
ratings = pd.read_csv(
    "../data/u1_base.txt", names=["user_id", "movie_id", "rating"], sep="\t"
)
generos = pd.read_csv("../data/genre.txt", names=["genre_id", "genre_name"], sep="\t")
all_genre = generos.genre_name.values.tolist()
all_genre = ["movie_id"] + all_genre + ["title"]
films_df = pd.read_csv("../data/items.txt",encoding="iso-8859-1" ,names=all_genre, sep="\t")

In [94]:
preferencias = []
for u in ratings.user_id.unique().tolist():
    best_genres = []
    for i in generos['genre_name'].tolist():
        puntos = genre_seen(films_df, ratings, u, i)
        if puntos > 2.5:
            best_genres.append((i, puntos))

    best_genres.sort(key=lambda a: a[1], reverse=True)
    best_genres = best_genres[:6]
    pref = []
    
    for genero in generos['genre_name'].tolist():
        add = False
        for bg in best_genres:
            if genero == bg[0]:
                add = True
                break
        if add:
            pref.append(1)
        else:
            pref.append(0)

    preferencias.append(pref)

In [97]:
preferencias = np.matrix(preferencias)

In [98]:
np.savez_compressed('preferencias', a=preferencias)

In [7]:
pref_loaded = np.load('preferencias.npz')

In [37]:
pref_dg = get_dg_pref(grupos_demograficos, generos, films_df, ratings)

In [61]:
pref_dg = np.load('preferencias_demografico.npz')

In [76]:
pref_hyb = []
for u in ratings.user_id.unique().tolist():
    pref = np.round(np.sum(np.matrix([pref_loaded['a'][u-1], pref_dg['a'][grupos_demograficos[u]-1]]), axis=0)/2, 0).tolist()[0]
    pref_hyb.append(pref)

pref_hyb = np.matrix(pref_hyb)

In [85]:
pref_hyb = np.load('preferencias_hibrido.npz')

In [ ]:
puntuaciones={}
for user_id in users_df.user_id.unique().tolist():
    puntuaciones[user_id] = {}
    print(user_id)
    dem_group = grupos_demograficos[user_id]
    for movie_id in films_df.movie_id.unique().tolist():
        v, r, c = movie_votes_demographic(grupos_demograficos, ratings, movie_id, dem_group)
        puntuaciones[user_id][movie_id] = v/((v+5)*r) + ((5/(5+v)) * c)

In [108]:
obtener_vecinos(pref_loaded['a'], 3, 3)

([58, 47, 3], [17, 17, 17])

In [96]:
pref_dg['a']

array([[0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0],
       [0,

In [1]:
from recomendador import Recomendador
import numpy as np

In [2]:
recom = Recomendador(data_path='../data/')
recom.load_grupos_demograficos()

In [3]:
recom.load_preferencias(path='../data/preferencias.npz', path_dg='../data/preferencias_demografico.npz', path_hyb='../data/preferencias_hibrido.npz')

In [4]:
recom.obtener_recomendacion_dg(2, 5)

[120, 169, 273, 316, 418, 460, 544, 602, 629, 713, 733, 797, 810, 914]
[210, 684, 660, 655, 651, 137, 566, 528, 509, 467, 423, 276, 307, 277, 276, 275, 268, 311, 241, 100, 19, 531, 14, 137, 651, 124, 716, 100, 923, 20, 258, 288, 19, 14, 344, 333, 1119, 899, 732, 693, 238, 223, 515, 204, 100, 98, 313, 87, 86, 69, 64, 56, 50, 23, 22, 15, 12, 50, 117, 294, 127, 302, 111, 117, 173, 199, 197, 194, 193, 187, 182, 174, 172, 127, 162, 153, 147, 144, 137, 135, 132, 582, 181, 257, 515, 50, 134, 213, 1067, 97, 342, 258, 300, 339, 331, 333, 328, 326, 321, 313, 606, 301, 684, 268, 300, 50, 181, 294, 289, 781, 879, 211, 269, 191, 286, 402, 276, 427, 778, 258, 155, 275, 515, 781, 902, 259, 304, 200, 304, 643, 207, 196, 195, 257, 15, 202, 881, 358, 323, 55, 878, 92, 678, 286, 197, 371, 9, 988, 338, 880, 331, 678, 160, 58, 538, 149, 234, 243, 658, 762, 744, 713, 676, 459, 729, 324, 237, 1109, 302, 269, 272, 302, 293, 310, 315, 290, 347, 286, 285, 273, 242, 107, 116, 224, 221, 924, 950, 1011, 319, 258, 

[[235, 0.5389533180270306],
 [9665, 0.5389533180270306],
 [89, 0.5125357560547048],
 [9386, 0.5125357560547048],
 [1633, 0.48468043436556224]]

In [5]:
recom.films_df

,movie_id,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,...,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,title,tmdb_id
0,1,0,0,0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,Toy Story (1995),862
1,2,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,GoldenEye (1995),710
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,Four Rooms (1995),5
3,4,0,1,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,Get Shorty (1995),8012
4,5,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,1,0,0,Copycat (1995),1710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,Mat' i syn (1997),44361
1678,1679,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,B. Monkey (1998),2923
1679,1680,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,Sliding Doors (1998),10215
1680,1681,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,You So Crazy (1994),38129


In [11]:
recom.obtener_recomendacion_dg(7,5)

[[16052, 4.084243106762175],
 [829, 4.727964586523274],
 [85, 4.6210008798330895],
 [7300, 4.6210008798330895],
 [27265, 4.5115018709314745]]

In [11]:
recom.preferencias_coop[1]

array([0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])

In [34]:
np.append(recom.preferencias_coop, [[0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0]], axis=0)[943]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])

In [12]:
recom.get_user_films(1)

[1,
 2,
 3,
 4,
 5,
 7,
 8,
 9,
 11,
 13,
 15,
 16,
 18,
 19,
 21,
 22,
 25,
 26,
 28,
 29,
 30,
 32,
 34,
 35,
 37,
 38,
 40,
 41,
 42,
 43,
 45,
 46,
 48,
 50,
 52,
 55,
 57,
 58,
 59,
 63,
 66,
 68,
 71,
 75,
 77,
 79,
 83,
 87,
 88,
 89,
 93,
 94,
 95,
 99,
 101,
 105,
 106,
 109,
 110,
 111,
 115,
 116,
 119,
 122,
 123,
 124,
 126,
 127,
 131,
 133,
 135,
 136,
 137,
 138,
 139,
 141,
 142,
 144,
 146,
 147,
 149,
 152,
 153,
 156,
 158,
 162,
 165,
 166,
 167,
 168,
 169,
 172,
 173,
 176,
 178,
 179,
 181,
 182,
 187,
 191,
 192,
 194,
 195,
 197,
 198,
 199,
 203,
 204,
 205,
 207,
 211,
 216,
 217,
 220,
 223,
 231,
 234,
 237,
 238,
 239,
 240,
 244,
 245,
 246,
 247,
 249,
 251,
 256,
 257,
 261,
 263,
 268,
 269,
 270,
 271,
 274]

In [13]:
recom.get_user_film_rate(1,2)

3